In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

In [ ]:
df=pd.read_csv("../input/prediction-of-loan-status/data.csv")
df.head()

In [ ]:
df.shape

# Covert to data time object

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data Visualization and Preprocessing

In [ ]:
df['loan_status'].value_counts()

In [ ]:
bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:

bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:Features Selection/extraction
Lets look at the day of the week people get the loan

In [ ]:

df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

# Convert Categorical features to numerical values

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

# One Hot Encoding

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

In [ ]:

df[['Principal','terms','age','Gender','education']].head()

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

# Feature selection

In [ ]:

X = Feature
X[0:5]

In [ ]:
y = df['loan_status'].values
y[0:5]

# Normalize Data

In [ ]:

X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Model Training

**K Nearest Neighbor(KNN)**

In [ ]:
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split( X, y, test_size=0.2, random_state=4)

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn import metrics

k_acc= []

k = 1
for k in range(1, 20):
   
    neigh = KNeighborsClassifier(n_neighbors = k).fit(X_trainset,y_trainset)

    yhat = neigh.predict(X_testset)
  
    print("Test set Accuracy: ", metrics.accuracy_score(y_testset, yhat), " - k =", k)
    k_acc.append(metrics.accuracy_score(y_testset, yhat))
    
plt.plot(k_acc)
plt.xlabel('k')
plt.ylabel('accuracy')
plt.show()

From here,you can see best test accuracy is with k=7 or 8.

Let k=7

In [ ]:
# F1 SCORE AND JACCARD SCORE FOR KNN WITH 7 NEIGBOURS
from sklearn.metrics import f1_score
# from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss

neigh = KNeighborsClassifier(n_neighbors = 7).fit(X_trainset,y_trainset)
knn_pred=neigh.predict(X_testset)
print("F1-SCORE:: ",f1_score( y_testset, knn_pred, average='weighted'))
print("Logloss score: ", log_loss(y_testset,neigh.predict_proba(X_testset)))

**Decision Tree**

In [ ]:
# Import 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=4)
# Print the shape of X_trainset and y_trainset. Ensure that the dimensions match
print("X-trainset-shape:: ", X_trainset.shape)
print("Y-trainset-shape:: ", y_trainset.shape)
loanTree = DecisionTreeClassifier(criterion="entropy",max_depth=6)
loanTree # SHOWS DEFAULT PARAMS

In [ ]:

loanTree.fit(X_trainset,y_trainset)

In [ ]:
predTree = loanTree.predict(X_testset)
# Lets see the Output of 5 results
print ("PREDICTED -- >", predTree [0:10])
print ("ACTUAL --> ", y_testset [0:10])

In [ ]:

from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

print("F1_score: ",f1_score(y_testset, predTree,average='weighted'))

**Support Vector Machine**

In [ ]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split( X, y, test_size=0.2, random_state=4)

Linear--->

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_trainset, y_trainset) 
yhat = clf.predict(X_testset)
print("SVM linear kernel: accuracy ", metrics.accuracy_score(y_testset, yhat), " \n F1: ", f1_score(y_testset, yhat, average='weighted'),  )

Polynomial---->

In [ ]:

clf = svm.SVC(kernel='poly')
clf.fit(X_trainset, y_trainset) 
yhat = clf.predict(X_testset)
print("SVM polynomial kernel: accuracy ", metrics.accuracy_score(y_testset, yhat), " \nF1: ", f1_score(y_testset, yhat, average='weighted'))

RBF----->

In [ ]:

clf = svm.SVC(kernel='rbf')
clf.fit(X_trainset, y_trainset) 
yhat = clf.predict(X_testset)
print("SVM rbf kernel: accuracy ", metrics.accuracy_score(y_testset, yhat), " \nF1: ", f1_score(y_testset, yhat, average='weighted'))

Sigmoid--->


In [ ]:

clf = svm.SVC(kernel='sigmoid')
clf.fit(X_trainset, y_trainset) 
yhat = clf.predict(X_testset)
print("SVM sigmoid kernel: accuracy ", metrics.accuracy_score(y_testset, yhat), " \nF1: ", f1_score(y_testset, yhat, average='weighted'))

# Logistic Regression

Newton-cg solver--->

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='newton-cg').fit(X_trainset,y_trainset)
yhat_lr = LR.predict(X_testset)
print("LR newton-cg solver: accuracy ", metrics.accuracy_score(y_testset, yhat_lr), " \nF1: ", f1_score(y_testset, yhat_lr, average='weighted'))
print("Logloss score: ", log_loss(y_testset,LR.predict_proba(X_testset)))

Liblinear solver---->

In [ ]:
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_trainset,y_trainset)
yhat_lr = LR.predict(X_testset)
print("LR liblinear solver: accuracy ", metrics.accuracy_score(y_testset, yhat_lr), " \nF1: ", f1_score(y_testset, yhat_lr, average='weighted'))
print("Logloss score: ", log_loss(y_testset,LR.predict_proba(X_testset)))


Saga solver----->

In [ ]:
LR = LogisticRegression(C=0.01, solver='saga').fit(X_trainset,y_trainset)
yhat_lr = LR.predict(X_testset)
print("LR saga solver: accuracy ", metrics.accuracy_score(y_testset, yhat_lr), " \nF1: ", f1_score(y_testset, yhat_lr, average='weighted'))
print("Logloss score: ", log_loss(y_testset,LR.predict_proba(X_testset)))

# REPORT

In [ ]:

table = [["KNN",0.7765,0.4671],["Decision Tree",0.6466,'NA'],
...          ["SVM",0.6914,'NA'],["LogisticRegression",0.6914,0.4920]]

from tabulate import tabulate
from IPython.display import HTML
HTML(tabulate(table, headers= ['Algorithm', 'F1-score','LogLoss'], tablefmt='html'))